# セマンティック カーネル例 (日本語版)

このノートブックではメタ認知の基礎概念を踏まえ、Semantic Kernel を用いた最小構成の呼び出し手順を段階的に確認します。コードセルごとに目的を先に示し、その後で実行内容を説明する形にしています。必要に応じてご自身のキーや設定値を差し替えてご利用ください。

### このノートブック学べる事
本ノートブックは Lesson 09 全体像のうち基礎的な部分を体験できるシンプルな構成です。以下の要素を実装しています。
- 計画(初期指示): system 指示で質問タイミングや提示形式の方針を明示しています。
- ツール利用: `get_destinations` と `get_flight_times` を登録し、モデルの Function Calling を介して呼び出しています。
- 出力後処理: どの関数がどの引数で呼ばれ結果が何だったかを、クリックで開閉できる詳細表示 (HTML の details/summary) に整理しています。
- ストリーミング観察: 応答生成を逐次受け取り、将来の“評価/反射”フック挿入の観測点を確保しています。

これらは後で自己修正や戦略切替を拡張するための土台となる要素です。

## 目的と流れ
このノートブックでは環境変数の設定、カーネル初期化、シンプルなプロンプト実行、関数拡張という順序でシンプルな構成を確認します。各ステップで「何を準備し何が得られるか」を簡単にではありますが示し、後続の再ランキングや戦略切替のための準備をします。
> 必要に応じて組織のポリシーに合わせてキー管理方法を置き換えてください。

In [1]:
# セル1: 基本インポートと SDK 読み込み
import json, os
from typing import Annotated
from dotenv import load_dotenv
from openai import AsyncOpenAI
from semantic_kernel import Kernel  # 追加: Kernel 本体
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.functions import kernel_function

In [ ]:
# セル2: 旅行先/フライト時間プラグイン
class DestinationsPlugin:
    """休暇用の目的地一覧とフライト時間を提供するプラグイン。"""

    # 日本語入力に対応するための日→英マッピング（簡易）
    _JA_TO_EN = {
        '東京': 'Tokyo',
        'パリ': 'Paris',
        'ベルリン': 'Berlin',
        'バルセロナ': 'Barcelona',
        'ニューヨーク': 'New York',
    }

    @kernel_function(description="休暇用の目的地一覧を返します")
    def get_destinations(self) -> Annotated[str, "目的地リスト"]:
        """目的地候補を改行区切りの一つの文字列として返します。
        NOTE: 元のコードはシングルクォートの配置が崩れて構文エラーになる状態でしたので、
        隣接文字列連結の正しい形に修正し、明示的に改行を含めています。"""
        return (
            "Barcelona, Spain\n"
            "Paris, France\n"
            "Berlin, Germany\n"
            "Tokyo, Japan\n"
            "New York, USA"
        )

    @kernel_function(description="指定した目的地の利用可能なフライト時間を返します")
    def get_flight_times(self, destination: Annotated[str, "フライト時間を調べたい都市 (日本語/英語どちらでも可)"]) -> Annotated[str, "フライト時間一覧"]:
        # flight_times: 都市ごとのサンプル時刻。実運用では外部APIやキャッシュ層へ置き換えます。
        flight_times = {
            'Barcelona': ['08:30 AM', '02:15 PM', '10:45 PM'],
            'Paris': ['06:45 AM', '12:30 PM', '07:15 PM'],
            'Berlin': ['07:20 AM', '01:45 PM', '09:30 PM'],
            'Tokyo': ['11:00 AM', '05:30 PM', '11:55 PM'],
            'New York': ['05:15 AM', '03:00 PM', '08:45 PM']
        }
        # 入力が "Paris, France" / "東京" などの場合でも都市名部分だけを抽出し、日本語なら英語へ正規化します。
        city_raw = destination.split(',')[0].strip()
        city_en = self._JA_TO_EN.get(city_raw, city_raw)
        if city_en in flight_times:
            # 表示時は元入力が日本語なら日本語表記を残しつつ英語キーで検索
            label = city_raw if city_raw != city_en else city_en
            return f'フライト時間 ({label}): ' + ', '.join(flight_times[city_en])
        return f'{city_raw} のフライト情報は見つかりません。'

In [3]:
# セル3: 環境変数とクライアント初期化
load_dotenv()
client = AsyncOpenAI(
    api_key=os.getenv('GITHUB_TOKEN'),  # NOTE: リポジトリ環境に合わせて .env で設定
    base_url='https://models.inference.ai.azure.com/',
)
chat_completion_service = OpenAIChatCompletion(ai_model_id='gpt-4o-mini', async_client=client)
print('# 初期化完了')

# 初期化完了


In [7]:
# セル4: エージェント定義とストリーミング実行 (都市名は日本語化しています)

# 目的: ChatCompletionAgent にツール(DestinationsPlugin)を登録し、ユーザー入力をストリーミングで処理する最小例を示します。
# 追加: 英語地名を応答表示時に日本語へ差し替えるシンプルなポストプロセス。

# エージェントの system 指示 (方針とメタ認知的自己修正の簡潔ルール)
AGENT_NAME = 'TravelAgent'
AGENT_INSTRUCTIONS = (
    "あなたはフライト予約と旅行アクティビティ提案を行う支援エージェントです。"\
    "提供ツール:\n"\
    "1. get_destinations: 選択可能な旅行先一覧。\n"\
    "2. get_flight_times: 指定都市の利用可能フライト時間。\n\n"\
    "ポリシー:\n"\
    "- 初回に希望フライト時間が無ければ一度だけ尋ねる。\n"\
    "- 一度取得した嗜好は customer_preferences に保持し再質問しない。\n"\
    "- 提案時は活用した嗜好を明示。\n"\
    "- 明らかに非現実的なフライト時間を出した場合は自分で修正し再提案 (メタ認知)。\n"\
    "- 出力形式: フライト時間は箇条書き。\n"\
    "目的: ユーザーの嗜好へ適応し効率的な旅行計画を支援する。"
)

# 英語→日本語 地名マッピング
_CITY_JA_MAP = {
    'Barcelona': 'バルセロナ',
    'Paris': 'パリ',
    'Berlin': 'ベルリン',
    'Tokyo': '東京',  # そのまま
    'New York': 'ニューヨーク',
    'Spain': 'スペイン',
    'France': 'フランス',
    'Germany': 'ドイツ',
    'USA': 'アメリカ'
}

def _to_ja(text: str) -> str:
    # 単純置換 (厳密な形態素処理は不要な想定)
    for en, ja in _CITY_JA_MAP.items():
        text = text.replace(en, ja)
    return text

# Kernel 構築とサービス登録
kernel = Kernel()
kernel.add_service(chat_completion_service)

# プラグインインスタンス登録
plugin = DestinationsPlugin()
kernel.add_functions(
    functions=[plugin.get_destinations, plugin.get_flight_times],
    plugin_name="travel"
)

# ChatCompletionAgent 生成
agent = ChatCompletionAgent(
    name=AGENT_NAME,
    kernel=kernel,
    instructions=AGENT_INSTRUCTIONS,
)

# 会話スレッド (履歴管理)
conversation_thread = ChatHistoryAgentThread()

from IPython.display import display, HTML

async def main():
    user_inputs = [
        'おすすめの旅行先を教えてください',
        '東京のフライト時間はありますか',
    ]
    html_output = ''
    conv_thread = conversation_thread  # 局所変数に保持 (再束縛で UnboundLocalError を避ける)
    for user_input in user_inputs:
        html_output += (
            f'<div style="margin-bottom:10px">'
            f'<div style="font-weight:bold">User:</div>'
            f'<div style="margin-left:20px">{user_input}</div></div>'
        )
        full_response = []
        function_calls = []
        current_function_name = None
        argument_buffer = ''
        async for response in agent.invoke_stream(messages=user_input, thread=conv_thread):
            # response.thread を返す実装でも新しい参照があれば更新 (存在チェック型安全)
            if hasattr(response, 'thread') and response.thread is not None:
                conv_thread = response.thread
            for item in list(response.items):
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed, ensure_ascii=False)
                        except Exception:
                            pass
                        function_calls.append(
                            f'Calling function: {current_function_name}({formatted_args})'
                        )
                        current_function_name = None
                        argument_buffer = ''
                    function_calls.append('Function Result: ' + _to_ja(str(item.result)))
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(_to_ja(item.text))
        if function_calls:
            html_output += (
                '<div style="margin-bottom:10px">'
                '<details><summary style="cursor:pointer; font-weight:bold; color:#0066cc;">Function Calls (click)</summary>'
                '<div style="margin:10px; padding:10px; background-color:#f8f8f8; border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;">'
                + '\n'.join(function_calls) + '</div></details></div>'
            )
        html_output += (
            f'<div style="margin-bottom:20px">'
            f'<div style="font-weight:bold">{AGENT_NAME}:</div>'
            f'<div style="margin-left:20px; white-space:pre-wrap">{''.join(full_response)}</div></div><hr>'
        )
    display(HTML(html_output))

await main()

In [8]:
# セル6: 継続会話 (追加例)
# 目的: 既存会話スレッドに追加質問を行い履歴継続とツール利用を確認します。

async def continue_chat():
    follow_ups = ['次はパリ行きフライトを予約したい']
    from IPython.display import display, HTML as _HTML
    html_output = ''
    conv_thread = conversation_thread  # セル4で作成済みのスレッドを参照
    for user_input in follow_ups:
        html_output += (
            f'<div style="margin-bottom:10px">'
            f'<div style="font-weight:bold">User:</div>'
            f'<div style="margin-left:20px">{user_input}</div></div>'
        )
        full_response = []
        function_calls = []
        current_function_name = None
        argument_buffer = ''
        async for response in agent.invoke_stream(messages=user_input, thread=conv_thread):
            if hasattr(response, 'thread') and response.thread is not None:
                conv_thread = response.thread
            for item in list(response.items):
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed, ensure_ascii=False)
                        except Exception:
                            pass
                        function_calls.append(
                            f'Calling function: {current_function_name}({formatted_args})'
                        )
                        current_function_name = None
                        argument_buffer = ''
                    function_calls.append('Function Result: ' + _to_ja(str(item.result)))
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(_to_ja(item.text))
        if function_calls:
            html_output += (
                '<div style="margin-bottom:10px">'
                '<details><summary style="cursor:pointer; font-weight:bold; color:#0066cc;">Function Calls (click)</summary>'
                '<div style="margin:10px; padding:10px; background-color:#f8f8f8; border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;">'
                + '\n'.join(function_calls) + '</div></details></div>'
            )
        html_output += (
            f'<div style="margin-bottom:20px">'
            f'<div style="font-weight:bold">{AGENT_NAME}:</div>'
            f'<div style="margin-left:20px; white-space:pre-wrap">{''.join(full_response)}</div></div><hr>'
        )
    display(_HTML(html_output))

await continue_chat()

### このノートブックに含まれていない（応用で拡張する）要素
以下の高度な要素は学習負荷を抑えるため本ノートブックにはまだ含めていません。`translations/ja/09-metacognition/README.md` と Microsoft Learn の関連モジュールを参考に、応用編として段階的に追加してみてください。
- メタ認知ループ（戦略評価と自己観察の反射ステップ）
- 自己修正型RAG（再検索→再生成の評価駆動ループ）
- 環境情報管理（嗜好・回避履歴・メトリクスの永続化）
- コード生成 / 動的ツール組込み（生成コードの安全評価を含めます）
- SQL活用（構造化クエリによる評価指標・網羅性メトリクス取得）

推奨拡張順序: 1) 追加の RAG 取得 + 再ランキング 2) 評価関数による自己修正ループ 3) 環境モデルの永続化 4) メタ認知ループ導入 5) SQL 指標計測とコード生成統合 です。

学習を進める際は小さな計測（成功率・再質問率・重複提案率）を先に入れてからアルゴリズム拡張を行うことで問題切り分けが容易になります。

##### 参考: Microsoft Learn 関連モジュール / ドキュメント
以下は拡張実装時に参照しやすい公式ドキュメント例です。
- Semantic Kernel 概要: https://learn.microsoft.com/semantic-kernel/overview
- Function Calling の使い方: https://learn.microsoft.com/azure/ai-services/openai/how-to/function-calling
- RAG の概要 (Azure AI Search): https://learn.microsoft.com/azure/search/retrieval-augmented-generation-overview
- RAG クイックスタート: https://learn.microsoft.com/azure/search/search-get-started-rag
- 生成 AI 応答評価 (評価の考え方): https://learn.microsoft.com/azure/well-architected/ai/test
- モデル応答品質の評価チュートリアル (.NET 例): https://learn.microsoft.com/dotnet/ai/evaluation/evaluate-ai-response
- セキュアな拡張とシステム メッセージ設計指針: https://learn.microsoft.com/azure/ai-services/openai/concepts/system-message

必要に応じて最新の Semantic Kernel ドキュメントや追加モジュールを検索し内容を補強してください。